### Black Scholes Option Pricing

In [6]:
%%file BS_Vanilla.m

function prices = BS_Vanilla(C_P, S, X, r, T, vol, q)
    
    if nargin < 7, q=0;
    end
    
    d1 = (log(S/X)+(r-q+1/2*vol^2)*T)/(vol*sqrt(T));
    d2 = d1- vol*sqrt(T);
    
    if C_P == 'C'
        price = S*exp(-q*T).*normcdf(d1) - X*exp(-r*T)*normcdf(d2);
    elseif C_P == 'P'
        price = -S*exp(-q*T).*normcdf(-d1) + X*exp(-r*T)*normcdf(-d2);
    end

Created file '/Users/changsoo/Desktop/논문/Matlab/BS_Vanilla.m'.


In [17]:
BS_Barrier('C', 100, 100, 0.05, 0.5, 0.3, 'uo', 130, 0.01)


ans =

    2.8481

